In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

xml_patent_data = open('FILEPATH HERE').read() #need to add file path to clean xml data

def patent_xml2df(xml_data):
    root = ET.XML(xml_data)
    all_records = []
    for patent in root:
        record = {}
        for element in patent[0]:
            if element.tag == 'publication-reference':
                for item in element[0]:
                    if item.tag == 'date':
                        record['grant_date'] = item.text
                    elif item.tag in ('country','doc-number'):
                        record[item.tag] = item.text
            elif element.tag == 'application-reference':
                record['application_type'] = element.get('appl-type')
                for item in element[0]:
                    if item.tag == 'date':
                        record['filing_date'] = item.text
                    elif item.tag == 'doc-number':
                        record['application_number'] = item.text
            elif element.tag == 'figures':
                for item in element:
                    record[item.tag] = item.text
            elif element.tag in ('number-of-claims','invention-title'):
                record[element.tag] = element.text
                
        all_records.append(record)
    return pd.DataFrame(all_records)

subset_df = patent_xml2df(xml_patent_data)

xml_patent_data_2 = open('./patent_full_clean.xml').read()

full_df = patent_xml2df(xml_patent_data_2)